In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib agg

import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
    
sys.path.append("../modules")
    
from INOSerial import Arduino
from HPIB4155 import HP4155
from HPIB_plot import Plot, PlotVgs, PlotVp, CalcIsSat, SecDer, Plot2P
from IPython.display import clear_output, display
from os import makedirs
from time import sleep
from datetime import datetime, timedelta

def WriteLog(msg, path, mode='a', end='\n', output=True):
    makedirs(path, exist_ok=True)
    with open(path, mode) as logfile:
        logfile.write(msg+end)
    if output:
        print(f'{msg}{end}', end='')

HP=HP4155('GPIB0::17', debug=False)
HP.SetIntTime("LONG")

INO=Arduino('COM3')
print(INO.ask('*'.encode()))

HEWLETT-PACKARD,4155A,0,01.04:01.04:01.00
InoMatrix



In [2]:
params = {'Vmin' : 0,
'Vmax' : 1.5,
'Vstep' : 0.02,

'Vgmin' : 0.6,
'Vgmax' : 1.4,
'Vgstep' : 0.2,

'Vd' : 0.025,
'temp' : 3.338,
'HiPot' : False}

path = 'C:/Users/Zucchi/Documents/Medidas/241007 - TN4 TN7 TN12 TP11/Cooldown/'
min_wait=30

In [13]:
freq_hour=2
duration_hours=48

DeviceList = ['TP2', 'CB1', 'TN12', 'TN4', 'DN1', '']
# DeviceList = ['', '', '', '', 'DN1', '']

prog_bar=display('',display_id=True)

for i in range(int(duration_hours*freq_hour)):
        
    while (datetime.now().time().minute+1)%(60/freq_hour):       
        prog_bar.update(datetime.now().strftime("%H:%M:%S"))
        sleep(0.5)

    clear_output(wait=True)

    prog_bar=display('',display_id=True)
    
    start=datetime.now()
    WriteLog(f"#  Measurement {i+1} - {(start+timedelta(seconds=60)).strftime('%y%m%d %H%M')}\n", path)

    plt.close('all')

    HP.SetStop("OFF")

    for n, prefix in enumerate(DeviceList):
        print(INO.opench(n+1))

        ####################### Measure Diode
        #
        #
        if 'd' in prefix.lower():
            WriteLog(f"## Ch {n+1} {prefix}", path)
            pathp=path+prefix
            makedirs(pathp, exist_ok=True)
            HP.SetStop("COMP")
            HP.SetIntTime("MED")
            HP.SingleDiode(0, -1 if 'n' in prefix.lower() else 1, -0.01 if 'n' in prefix.lower() else 0.01, 'SMU2', 'SMU1', Comp=1e-3)
            while datetime.now().time().second == 57:       
                prog_bar.update(datetime.now().strftime("%H:%M:%S"))
                sleep(0.5)
            now=datetime.now().strftime("%y%m%d %H%M%S")
            Plot(HP.SingleSave(f"{pathp}/{now}.csv", timeout=30, real=True), "Vf", "If")
            rename(f"{pathp}/{now}.csv", f"{pathp}/{GetT()} - {now}.csv")
            rename(f"{pathp}/{now}.png", f"{pathp}/{GetT()} - {now}.png")
            
            for t in range(min_wait):
                prog_bar.update(f'{datetime.now().strftime("%H:%M:%S")} Waiting minimum {min_wait}s')
                sleep(0.5)
            HP.SetStop("OFF")
            HP.SetIntTime("LONG")
            print(INO.opench(0))
            WriteLog('', path)
            continue
        #
        #
        #######################
            
        ####################### Measure CrossBridge
        #
        #
        if 'c' in prefix.lower() or prefix.lower()=='short':
            WriteLog(f"## Ch {n+1} {prefix}", path)
            pathp=path+prefix
            makedirs(pathp, exist_ok=True)
            HP.Set2P(-1e-4, 1e-4, 50, SMUN='SMU1', SMUP='SMU2', Comp=1)
            while datetime.now().time().second != 57:       
                prog_bar.update(datetime.now().strftime("%H:%M:%S"))
                sleep(0.5)
            now=datetime.now().strftime("%y%m%d %H%M")
            Rshort=Plot2P(HP.SingleSave(f"{pathp}/2P - {now}.csv", timeout=30))
            rename(f"{pathp}/2P - {now}.csv", f"{pathp}/2P - {GetT()} - {now}.csv")
            rename(f"{pathp}/2P - {now}.png", f"{pathp}/2P - {GetT()} - {now}.png")
            for t in range(min_wait):
                prog_bar.update(f'{datetime.now().strftime("%H:%M:%S")} Waiting minimum {min_wait}s')
                sleep(0.5)
        
            WriteLog(f"Rshort={format(Rshort, '.2f')}", path)
            with open(f"{pathp}/RxT 2P.log", 'a') as RxT2p:
                RxT2p.write(f"{now},{format(Rshort, '.2f')}")
            print('')
        
            HP.Set4P(-5e-3, 5e-3, 50, Im='SMU1', Ip='SMU2', Comp=1)
            while datetime.now().time().second != 57:       
                prog_bar.update(datetime.now().strftime("%H:%M:%S"))
                sleep(0.5)
            now=datetime.now().strftime("%y%m%d %H%M")
            RCB=Plot2P(HP.SingleSave(f"{pathp}/4P - {now}.csv", timeout=30))
            rename(f"{pathp}/4P - {now}.csv", f"{pathp}/4P - {GetT()} - {now}.csv")
            rename(f"{pathp}/4P - {now}.png", f"{pathp}/4P - {GetT()} - {now}.png")
            
            for t in range(min_wait):
                prog_bar.update(f'{datetime.now().strftime("%H:%M:%S")} Waiting minimum {min_wait}s')
                sleep(0.5)
            WriteLog(f"RCB={format(RCB, '.2e')}", path)
            with open(f"{pathp}/RxT 4P.log", 'a') as RxT4p:
                RxT4p.write(f"{now},{format(RCB, '.2e')}")
            print('')
            WriteLog('', path)
            print(INO.opench(0))
            continue
        #
        #
        #######################

        ####################### Measure Transistor
        #
        #
        if 't' in prefix.lower():
            WriteLog(f"## Chn{n+1} - {prefix}", path)
            pathp=path+prefix
            makedirs(pathp, exist_ok=True)
            ptype='p' in prefix.lower()
            
            HP.SetVgs(params['Vmin'], params['Vmax'], params['Vstep'], params['Vd'], ptype=ptype)
            while datetime.now().time().second != 45:       
                prog_bar.update(datetime.now().strftime("%H:%M:%S"))
                sleep(0.5)
            now=datetime.now().strftime("%y%m%d %H%M")
            LIN = PlotVgs(HP.SingleSave(f'{pathp}/IdVgs - {now}.csv', timeout=30))
            rename(f'{pathp}/IdVgs - {now}.csv', f'{pathp}/IdVgs - {GetT()} - {now}.csv')
            rename(f'{pathp}/IdVgs - {now}.png', f'{pathp}/IdVgs - {GetT()} - {now}.png')
            for t in range(min_wait):
                prog_bar.update(f'{datetime.now().strftime("%H:%M:%S")} Waiting minimum {min_wait}s')
                sleep(0.5)
            WriteLog(f'Vth= {LIN} V', path)
            try:
                DGM=SecDer(f'{pathp}/IdVgs - {now}.csv')
                WriteLog(f'SecDer= {DGM} V', path)
            except:
                WriteLog(f'SecDer Fail', path)
                print('')
        
            if params['HiPot']:
                HP.SetVgs(params['Vmin'], params['Vmax'], params['Vstep'], params['Vmax'], ptype=ptype, sat=True)
                n, Ispec = CalcIsSat(HP.SingleSave(f'{pathp}/IdVgsSat - {now}.csv', timeout=30), float(GetT()))
                rename(f'{pathp}/IdVgsSat - {now}.csv', f'{pathp}/IdVgsSat - {GetT()} - {now}.csv')
                WriteLog(f"n={format(n, '.3f')}, Ispec={format(Ispec, '.3e')} A", path)
                print('')
                
                if Ispec != 0:
                    HP.SetVp(Ispec, params['Vmin'], params['Vmax'], 0.05, ptype=ptype)
                    VTO=PlotVp(HP.SingleSave(f'{pathp}/VpVg - {now}.csv', timeout=30))
                    WriteLog(f'VTO= {VTO} V', path)
                    print('')
                    
                HP.SetVds(params['Vmin'], params['Vmax'], params['Vstep'], params['Vgmin'], params['Vgmax'], params['Vgstep'], ptype=ptype)
                Plot(HP.SingleSave(f'{pathp}/IdVds - {now}.csv', timeout=30), 'Vd', 'Id')
                rename(f'{pathp}/IdVds - {now}.csv', f'{pathp}/IdVds - {GetT()} - {now}.csv')
                print('')
            WriteLog('', path)
            print(INO.opench(0))
            continue
        #
        #
        #######################
        
        print(INO.opench(0))
        sleep(2)
    WriteLog(f"Measurement {i+1} end. Duration: {int((datetime.now()-start).seconds/60)}:{(datetime.now()-start).seconds%60:02d}\n\n#######################\n", path)

'12:36:46'

#  Measurement 47 - 241010 1230

Open INO: ch1

## Chn1 - TP2
Set IdxVgs
Vg=(0, -1.5, -0.02), Vd=-0.025, Ilim=0.001
Starting IdxVgs. Duration: 34 s
Vth= -1.156 V
SecDer Fail


Closed
Open INO: ch2

## Ch 2 CB1
Set 2P - 12
I=(-0.0001, 0.0001), 50 Points
Starting 2P - 12. Duration: 7 s
Rshort=27.60

Set 4P
I=(-0.005, 0.005), 50 Points
Starting 4P. Duration: 20 s
RCB=3.68e-02


Closed
Open INO: ch3

## Chn3 - TN12
Set IdxVgs
Vg=(0, 1.5, 0.02), Vd=0.025, Ilim=0.001
Starting IdxVgs. Duration: 33 s
Vth= 0.54 V
SecDer Fail


Closed
Open INO: ch4

## Chn4 - TN4
Set IdxVgs
Vg=(0, 1.5, 0.02), Vd=0.025, Ilim=0.001
Starting IdxVgs. Duration: 35 s
Vth= 0.688 V
SecDer Fail


Closed
Open INO: ch5

## Ch 5 DN1
Diode=(0, -1, -0.01)
Starting Diode. Duration: 10 s
Closed

Open INO: ch6

Closed
Measurement 47 end. Duration: 6:12

#######################



KeyboardInterrupt: 

In [14]:
HP.close()
INO.close()